In [ ]:
 Import Train and Test Data 
source  = '/Users/Administrator/Downloads/' 
datafile = pd.read_csv(source + 'bitcoind.csv')
#columns = ['Basic_Size','Gr','Target_Pull_Size']
bitcoindf = pd.DataFrame(datafile)
bitcoindf.tail(50)

In [ ]:
#import modules
from datetime import date
#convert column "date" into datetime format
bitcoindf['Date'] = pd.to_datetime(bitcoindf['Date'])
#apply weeks of year

In [ ]:
bitcoindf=bitcoindf.set_index('Date')
bitcoindf.head()

In [ ]:
#x and y axes input data
y = bitcoindf['Price']

In [ ]:
# Plot the original data, the trend, the seasonality, and the residuals 

from statsmodels.tsa.seasonal import seasonal_decompose
data=seasonal_decompose(y,model='additive')
data.plot()

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss(y,nlags='legacy')

In [ ]:
y_log=np.log(y)

In [ ]:
y_log.plot()

In [ ]:
kpss(y_log.diff().dropna())

In [ ]:
# Make a function called evaluate_arima_model to find the MSE of a single ARIMA model 
def evaluate_arima_model(data, arima_order):
    # Needs to be an integer because it is later used as an index.
    # Use int()
    split=int(len(data) * 0.8) 
    # Make train and test variables, with 'train, test'
    train, test = data[0:split], data[split:len(data)]
    past=[x for x in train]
    # make predictions
    predictions = list()
    for i in range(len(test)):#timestep-wise comparison between test data and one-step prediction ARIMA model. 
        model = ARIMA(past, order=arima_order)
        model_fit = model.fit(disp=0)
        future = model_fit.forecast()[0]
        predictions.append(future)
        past.append(test[i])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    # Return the error
    return error

In [ ]:
 #Make a function called evaluate_models to evaluate different ARIMA models with several different p, d, and q values.
def evaluate_models(dataset, p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None
    # Iterate through p_values
    for p in p_values:
        # Iterate through d_values
        for d in d_values:
            # Iterate through q_values
            for q in q_values:
                # p, d, q iterator variables in that order
                order = (p,d,q)
                try:
                    # Make a variable called mse for the Mean squared error
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    return print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

In [ ]:
# Now, we choose a couple of values to try for each parameter: p_values, d_values and q_values
# Fill in the blanks as appropriate
p_values = [x for x in range(0, 3)]
d_values = [x for x in range(0, 3)]
q_values = [x for x in range(0, 3)]

In [ ]:
# Finally, we can find the optimum ARIMA model for our data.
# Nb. this can take a while...!
import warnings
warnings.filterwarnings("ignore")
evaluate_models(y_log, p_values, d_values, q_values)

In [ ]:
p=2
d=0
q=0
model = ARIMA(y_log, order=(p,d,q))
model_fit = model.fit()
forecast = model_fit.forecast(24)

In [ ]:
import matplotlib as plt

from matplotlib import *
import sys
import pylab as pl
pl.figure(figsize =(17,5))
pl.plot(y_log.diff())
pl.plot(model_fit.predict(),color='red')

In [ ]:
import matplotlib.pyplot as plt
# Declare a variable called forecast_period with the amount of months to forecast, and
# create a range of future dates that is the length of the periods you've chosen to forecast
forecast_period = 24
date_range = pd.date_range(y.index[-1], periods = forecast_period, 
              freq='MS').strftime("%Y-%m-%d").tolist()

# Convert that range into a dataframe that includes your predictions

future_months = pd.DataFrame(date_range, columns = ['Month'])
future_months['Month'] = pd.to_datetime(future_months['Month'])
future_months.set_index('Month', inplace = True)
future_months['Prediction'] = forecast[0]
# Plot your future predictions
plt.figure(figsize=(15,10))
plt.plot(y)
plt.plot(np.exp(y_log.append(future_months['Prediction'])))
plt.show()
yforcast = np.exp(y_log.append(future_months['Prediction']))
#y_hats  = pd.DataFrame({'A' : yforcast})
#y_hats 
